In [1]:
import pandas as pd
import re
import seaborn as sns
import requests
import json
from pandas import json_normalize
import re

In [2]:
df = pd.read_csv('archive/ghg-emissions.csv')

In [3]:
df.head()

,Country/Region,unit,1990,1991,1992,1993,1994,1995,1996,1997,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,China,MtCO2e,2873.71,3023.38,3154.54,3377.79,3528.81,3918.30,3954.75,3963.61,...,9046.19,9872.37,10364.83,10690.04,11144.76,11155.76,11149.68,11207.66,11408.26,11705.81
1,United States,MtCO2e,5543.47,5501.92,5583.80,5691.26,5782.20,5843.90,6010.62,6258.47,...,5793.66,6041.59,5796.78,5550.87,5690.79,5711.15,5586.69,5676.92,5613.64,5794.35
2,India,MtCO2e,1009.44,1064.49,1096.71,1128.44,1174.78,1240.44,1289.63,1348.09,...,2467.78,2576.93,2610.32,2758.53,2816.49,2988.34,3002.17,3073.24,3202.82,3346.63
3,Russia,MtCO2e,2885.12,2788.70,2551.94,2294.62,2011.47,1930.88,1889.18,1755.59,...,1575.54,1690.93,1864.86,1886.41,1844.48,1841.26,1791.21,1918.26,1939.26,1992.08
4,Indonesia,MtCO2e,1257.10,1274.43,1295.01,1311.27,1327.87,1360.88,1179.00,2142.82,...,1483.61,1125.25,1675.72,1695.27,1636.17,2016.28,2070.24,1453.23,1466.05,1703.86


In [4]:
df.tail()

,Country/Region,unit,1990,1991,1992,1993,1994,1995,1996,1997,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
190,Nauru,MtCO2e,0.13,0.13,0.13,0.12,0.12,0.11,0.11,0.11,...,0.05,0.05,0.05,0.05,0.05,0.06,0.06,0.07,0.07,0.08
191,Tuvalu,MtCO2e,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,...,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.03,0.03
192,Niue,MtCO2e,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.00,0.00,0.00,0.01,0.01,0.01,0.01,0.01
193,Data source,CAIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,Data for,Countries/Regions: World. Sectors/Subsectors: ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Vemos que aquí las dos ultimas columnas, donde estan los nan, no son realmente paises, por ello podemos quitarlas, no se van a usar en ningún momento de este estudio

In [5]:
df.drop([193,194], axis=0, inplace=True)

In [6]:
df.tail()

,Country/Region,unit,1990,1991,1992,1993,1994,1995,1996,1997,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
188,Kiribati,MtCO2e,0.04,0.04,0.04,0.05,0.04,0.05,0.05,0.05,...,0.08,0.08,0.08,0.08,0.09,0.09,0.09,0.10,0.10,0.11
189,Cook Islands,MtCO2e,0.03,0.03,0.03,0.03,0.03,0.04,0.04,0.05,...,0.07,0.07,0.08,0.08,0.08,0.08,0.08,0.09,0.09,0.10
190,Nauru,MtCO2e,0.13,0.13,0.13,0.12,0.12,0.11,0.11,0.11,...,0.05,0.05,0.05,0.05,0.05,0.06,0.06,0.07,0.07,0.08
191,Tuvalu,MtCO2e,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,...,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.03,0.03
192,Niue,MtCO2e,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.00,0.00,0.00,0.01,0.01,0.01,0.01,0.01


### Llamamos a la API

In [7]:
url = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country')

In [8]:
url.json()

[{'name': 'Afghanistan', 'iso3': 'AFG', 'iso2': 'AF', 'id': 466},
 {'name': 'Albania', 'iso3': 'ALB', 'iso2': 'AL', 'id': 582},
 {'name': 'Algeria', 'iso3': 'DZA', 'iso2': 'DZ', 'id': 520},
 {'name': 'American Samoa (U.S.)', 'iso3': 'ASM', 'id': 677},
 {'name': 'Andorra', 'iso3': 'AND', 'id': 675},
 {'name': 'Angola', 'iso3': 'AGO', 'iso2': 'AO', 'id': 544},
 {'name': 'Anguilla (U.K.)', 'iso3': 'AIA', 'id': 716},
 {'name': 'Antigua and Barbuda', 'iso3': 'ATG', 'iso2': 'AG', 'id': 583},
 {'name': 'Argentina', 'iso3': 'ARG', 'iso2': 'AR', 'id': 563},
 {'name': 'Armenia', 'iso3': 'ARM', 'iso2': 'AM', 'id': 584},
 {'name': 'Aruba (Neth.)', 'iso3': 'ABW', 'id': 674},
 {'name': 'Australia', 'iso3': 'AUS', 'iso2': 'AU', 'id': 585},
 {'name': 'Austria', 'iso3': 'AUT', 'iso2': 'AT', 'id': 586},
 {'name': 'Azerbaijan', 'iso3': 'AZE', 'iso2': 'AZ', 'id': 491},
 {'name': 'Bahamas, The', 'iso3': 'BHS', 'iso2': 'BS', 'id': 587},
 {'name': 'Bahrain', 'iso3': 'BHR', 'iso2': 'BH', 'id': 588},
 {'name':

## ¿Qué daotos vamos a querer buscar para después comprarlos con los datos de nuestro dataset?
#### Temperaturas y precipitaciones de entre los años 1980 y 1999 de EEUU, Alemania, China, India ,Rusia y reino Unido (potencias mundiales y/o paises altamente poblados)
#### Depués se buscarán los datos predictivos de años venideros de la temperatura y de las precipitaciones de estos mismos países

Todas las funciones de extración de datos y los loops de la limpieza de los dataframes finales se encuentran en el main.py
A continuación tan solo se va a mostrar el producto final.

In [89]:
df_oldtemp = pd.read_csv('old_temp.csv',index_col=0)
df_oldprec = pd.read_csv('old_prec.csv', index_col=0)
df_newtemp = pd.read_csv('prediction_temp.csv', index_col=0)
df_newprec = pd.read_csv('prediction_prec.csv', index_col=0)

In [90]:
df_oldtemp

,Annual_Data,Country
0,6.344341,USA
1,7.677910,Germany
2,4.626097,China
3,9.121408,UK
4,21.990156,India
5,-7.352788,Rusia


In [91]:
df_oldprec

,Annual_Data,Country
0,841.912621,USA
1,929.598637,Germany
2,808.955153,China
3,988.845497,UK
4,1018.582287,India
5,497.024078,Rusia


In [92]:
df_newtemp

,Annual_Data,Country
0,7.483289,USA
1,8.535085,Germany
2,5.728654,China
3,9.668244,UK
4,22.846838,India
5,-5.949377,Rusia


In [93]:
df_newprec

,Annual_Data,Country
0,859.000935,USA
1,949.684790,Germany
2,817.829902,China
3,1005.959857,UK
4,1018.861217,India
5,520.475287,Rusia



- df (el dataset original ya limpio)

- old_temp (el concatenado de las temperaturas de los años 80 hasta el 99)

- old_prec (el concatenado de las precipitaciones de los años 80 hasta el 99)

- new_temp (el concatenado de la predicción de las temperaturas desde el 2020 hasta el 2039)

- new_prec (el concatenado de la predicción de las precipitaciones desde el 2020 hasta el 2039)